In [1]:
import sys, os
from os import path
sys.path.append('/root/bdrisk/risk_project')
sys.path.append(path.dirname(path.dirname(path.dirname(os.getcwd()))))
# sys.path.append('C:\\Users\\Administrator\\Desktop\\风控产品\\risk_project')
# sys.path.append(path.dirname(path.dirname(path.dirname(os.getcwd()))))
from risk_models import *

In [2]:
child_task_id = '0001_00042'
org_code, param_json, basetime = read_log_table(child_task_id)

2021-07-02 16:31:11.098 | INFO     | risk_models.config.read_config.read_func:read_oracle:83 - Read Table successfully! , Total read time spent 0.602s


In [4]:
org_code = org_code
# 默认为None
rate_thr_sup = json.loads(param_json)['rate_thr_sup']
rate_thr_inf = json.loads(param_json)['rate_thr_inf']
# 默认为 [1,-1]
score_crit = json.loads(param_json)['score_crit']

In [7]:

# 应加入公司ID筛选，默认取ISCURRENT为1的数据，
sql=f"""\
select * from {TableList.BD_RISK_DETAIL_STOCK_ST4.value} WHERE ISCURRENT = 1 AND ORG_CODE LIKE '{org_code}' \
"""     
MODEL_DETAIL = Read_Oracle().read_oracle(sql=sql,database='dbdm')
MODEL_DETAIL = MODEL_DETAIL.drop(columns = ['ID','ISCURRENT','LASTUPDATE'])

# 若阈值为空，则设为RATE的90分位数与10分位数
if pd.isnull(rate_thr_sup):
    rate_thr_sup = MODEL_DETAIL['RATE'].quantile(0.9)
else:
    rate_thr_sup = self.rate_thr_sup

if pd.isnull(rate_thr_inf):
    rate_thr_inf = MODEL_DETAIL['RATE'].quantile(0.1)
else:
    rate_thr_inf = rate_thr_inf

MODEL_DETAIL['RISK_LABEL'] = '数据正常'
MODEL_DETAIL['SCORE'] = score_crit[0]

data_length = MODEL_DETAIL.shape[0]




2021-07-02 16:32:57.647 | INFO     | risk_models.config.read_config.read_func:read_oracle:83 - Read Table successfully! , Total read time spent 0.248s


In [8]:
MODEL_DETAIL

,ORG_CODE,ORDER_NO,COP_G_NO,SIGN_DT,DELIV_FLAG,QTY_ORDER,QTY_DELIV,QTY_STOCK,PRICE_ORDER,PRICE_DELIV,PRICE_DELIV_EST,PRICE_STOCK,RATE,DAY_DIFF,CHECK_TIME,RISK_LABEL,SCORE
0,91310000132612172J,TEST_NO_000190,T_123000190,2020-11-11,None,60.0,80.0,46.0,2957.1,2982.0,3942.80000,2654.0,-0.32687,233,2021-07-02 16:26:59,数据正常,0
1,91310000132612172J,TEST_NO_000191,T_123000191,2020-07-04,None,55.0,53.0,58.0,3187.4,3602.0,3071.49455,3376.0,0.09914,363,2021-07-02 16:26:59,数据正常,0
2,91310000132612172J,TEST_NO_000192,T_123000192,2020-09-18,None,55.0,114.0,75.0,2874.5,2595.0,5958.05455,2691.0,-0.54834,287,2021-07-02 16:26:59,数据正常,0
3,91310000132612172J,TEST_NO_000193,T_123000193,2020-02-05,None,61.0,85.0,30.0,3170.3,2677.0,4417.63115,2525.0,-0.42843,513,2021-07-02 16:26:59,数据正常,0
4,91310000132612172J,TEST_NO_000195,T_123000195,2020-08-03,None,50.0,91.0,73.0,2956.1,2712.0,5380.10200,2639.0,-0.50949,333,2021-07-02 16:26:59,数据正常,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,91310000132612172J,TEST_NO_001187,T_123001187,2020-06-07,None,58.0,27.0,51.0,3042.3,2403.0,1416.24310,1909.0,0.34793,390,2021-07-02 16:26:59,数据正常,0
2460,91310000132612172J,TEST_NO_001188,T_123001188,2020-10-19,None,67.0,136.0,71.0,3035.8,3185.0,6162.22090,2693.0,-0.56298,256,2021-07-02 16:26:59,数据正常,0
2461,91310000132612172J,TEST_NO_001189,T_123001189,2020-09-23,None,65.0,28.0,100.0,3060.1,3686.0,1318.19692,4090.0,2.10272,282,2021-07-02 16:26:59,数据正常,0
2462,91310000132612172J,TEST_NO_001190,T_123001190,2020-03-02,None,62.0,62.0,29.0,2829.4,3038.0,2829.40000,2919.0,0.03167,487,2021-07-02 16:26:59,数据正常,0


In [20]:
child_task_id = '0001_0005'
org_code, param_json, base_time = read_log_table(child_task_id)

2021-07-02 17:00:01.760 | INFO     | risk_models.config.read_config.read_func:read_oracle:83 - Read Table successfully! , Total read time spent 0.198s


In [10]:
# 异常数据每百条扣0.75分
MODEL_DETAIL.loc[MODEL_DETAIL['RATE']<rate_thr_inf,'RISK_LABEL'] = '入库价值过低'
MODEL_DETAIL.loc[MODEL_DETAIL['RATE']<rate_thr_inf,'SCORE'] = (score_crit[1]/data_length*100*0.75)

In [16]:
MODEL_DETAIL.loc[MODEL_DETAIL['RATE']>rate_thr_sup,'RISK_LABEL'] = '入库价值过高'
MODEL_DETAIL.loc[MODEL_DETAIL['RATE']>rate_thr_sup,'SCORE'] = (score_crit[1]/data_length*100*0.75)

In [17]:
MODEL_DETAIL.loc[MODEL_DETAIL['RATE']>rate_thr_sup,'SCORE'] = 0

In [21]:
p1=f"""select * from {TableList.BD_RISK_DETAIL_CREDIT_CR1.value} WHERE ISCURRENT = 1 AND ORG_CODE LIKE '{org_code}' """
CR_DETAIL = Read_Oracle().read_oracle(sql= p1, database = 'dbods')

2021-07-02 17:00:40.899 | WARNING  | risk_models.config.read_config.read_func:read_ods:28 - Sql cannot find table, please check database
2021-07-02 17:00:40.905 | INFO     | risk_models.config.read_config.read_func:read_oracle:83 - Read Table successfully! , Total read time spent 7.229s


In [22]:
p1

"select * from BD_RISK_DETAIL_CREDIT_CR1 WHERE ISCURRENT = 1 AND ORG_CODE LIKE '91310000132612172J' "

In [19]:
CR_DETAIL

,ID,ORG_CODE,ORG_NAME,CASE_ID,CASE_INFO,CASE_TIME,STATUS,ROLE,EVENT_TYPE,CHECK_TIME,ISCURRENT,LASTUPDATE
0,12,123123123,XXXXXXXXX,nan,nan,NaT,nan,nan,nan,2021-06-18 15:19:23,0,2021-06-24 16:43:39
1,13,91310115607344141Q,乔信国际物流（上海）有限公司,（2017）沪73民初713号,计算机软件开发合同纠纷,2017-10-27,民事一审结案方式:撤诉,被告,涉诉一审,2021-06-21 09:26:14,1,2021-06-21 09:26:18
2,14,91310115607344141Q,乔信国际物流（上海）有限公司,（2018）沪0101民初13055号,电信服务合同纠纷,2018-06-11,民事一审结案方式:撤诉,被告,涉诉一审,2021-06-21 09:26:14,1,2021-06-21 09:26:18
3,15,91310115607344141Q,乔信国际物流（上海）有限公司,（2019）沪0101民初9721号,电信服务合同纠纷,2019-04-16,民事一审结案方式:撤诉,被告,涉诉一审,2021-06-21 09:26:14,1,2021-06-21 09:26:18
4,18,123123123,XXXXXXXXX,None,None,NaT,None,None,None,2021-06-24 16:43:37,1,2021-06-24 16:43:39
5,17,123123123,XXXXXXXXX,nan,nan,NaT,nan,nan,nan,2021-06-24 16:38:50,0,2021-06-24 16:43:39
6,0,91310115607344141Q,乔信国际物流（上海）有限公司,（2017）沪73民初713号,计算机软件开发合同纠纷,2017-10-27,民事一审结案方式:撤诉,被告,涉诉一审,2021-06-17 16:08:40,0,2021-06-21 09:26:18
7,1,91310115607344141Q,乔信国际物流（上海）有限公司,（2018）沪0101民初13055号,电信服务合同纠纷,2018-06-11,民事一审结案方式:撤诉,被告,涉诉一审,2021-06-17 16:08:40,0,2021-06-21 09:26:18
8,2,91310115607344141Q,乔信国际物流（上海）有限公司,（2019）沪0101民初9721号,电信服务合同纠纷,2019-04-16,民事一审结案方式:撤诉,被告,涉诉一审,2021-06-17 16:08:40,0,2021-06-21 09:26:18
9,3,91310115607344141Q,乔信国际物流（上海）有限公司,（2017）沪73民初713号,计算机软件开发合同纠纷,2017-10-27,民事一审结案方式:撤诉,被告,涉诉一审,2021-06-17 16:08:40,0,2021-06-21 09:26:18
